<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/HW3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [227]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

---
# Write the linear regression hypothesis function in vector form;
напишіть функцію гіпотези лінійної регресії у векторному вигляді;
---

$$h(x) = w_0 + w_1*x$$
$$h(x_1,x_2) = w_0 + w_1*x_1 + w_2*x_2 $$
$$h(x_1,...,x_n) = w_0 + w_1*x_1 + ... + w_n*x_n $$
$$ h(x_i) = w0 + \sum_{i=1}^n w_n*x_n $$

$$ \vec{x} = \begin{pmatrix} 1 \\ x_1 \\ ..  \\ x_n   \end{pmatrix},
\vec{w} = \begin{pmatrix} w_0 \\ w_1 \\ ..\\ w_n \end{pmatrix}$$

$$ h(\vec{x})= \vec{x} \cdot \vec{w}  $$
<!-- $$h(\overrightarrow{x}) = \overrightarrow{w} \cdot  \overrightarrow{x}$$ -->

or
$$
h(\mathbf{X}) = \mathbf{w}^T \mathbf{X}
$$


In [228]:
def hypothesis(X, w):
  return np.dot(X, w)

---
# Create a function to calculate the loss function in vector form;
створіть функцію для обчислення функції втрат у векторному вигляді;
---

$$
Loss = \frac{1}{2n} \| Xw - y \|^2
$$
or
$$
J(w_0​,w_1, ... ,w_n)=\frac{1}{2m} \sum_{i=1}^{m}​(h(x^{(i)})−y^{(i)})^2  
$$



In [229]:
def loss_func(X, y, w):
    m = len(y)
    h = hypothesis(X, w)
    error = h - y
    loss = (1 / (2 * m)) * np.sum((error) ** 2)
    return loss

---
# Implement one step of gradient descent;
реалізуйте один крок градієнтного спуску;
---

градієнт функції втрат -
$$  
\nabla J(\vec{w}) =
\begin{pmatrix}
\frac{\partial J(\vec{w})}{\partial w_0} \\
\frac{\partial J(\vec{w})}{\partial w_1} \\
\frac{\partial J(\vec{w})}{\partial w_2} \\
.. \\
\frac{\partial J(\vec{w})}{\partial w_n} \\
\end{pmatrix}
=
\begin{pmatrix}
\frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})\\
\frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})x_1^{(i)} \\
\frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})x_2^{(i)} \\
.. \\
\frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})x_n^{(i)} \\
\end{pmatrix}
$$

$$
\vec{w}
=\vec{w}^{prev} − α \nabla J(\vec{w})
$$

m — кількість зразків (samples).

n — кількість ознак (features) у кожному зразку.

$$
X_{m,n}=
\begin{bmatrix}
 1&  x^{(1)}_{1}&  x^{(1)}_{2}&  …&  x^{(1)}_{n}  \\
 1&  x^{(2)}_{1}&  x^{(2)}_{2}&  …&  x^{(2)}_{n}  \\
 1&  x^{(3)}_{1}&  x^{(3)}_{2}&  …&  x^{(3)}_{n}  \\
 \vdots&   \vdots&   \vdots&   \ddots&  \vdots&   \\
 1&  x^{(m)}_{1}&  x^{(m)}_{2}&  …&  x^{(m)}_{n}  \\
\end{bmatrix}​​
$$

$$
∇J(\vec{w})=\frac{1}{m}​  X^T(X \vec{w} - \vec{y}​)
$$

In [230]:
def gradient_descent_step(X, y, w, learning_rate=0.001):
    m = len(y)
    h = hypothesis(X, w)
    error = h - y
    gradient = (1 / m) * np.dot(X.T, error)
    w -= learning_rate * gradient
    return w

In [231]:
def gradient_descent(X, y, w, num_iterations=1, learning_rate=0.001):
    loss_history = np.zeros(num_iterations)
    for i in range(num_iterations):
        w = gradient_descent_step(X, y, w, learning_rate)
        loss_history[i] = loss_func(X, y, w)
    return w, loss_history



---
# Find the best parameters w for the dataset predicting the price of a house depending on the area, number of bathrooms, and number of bedrooms;

знайдіть найкращі параметри w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;
  ---


# **`Data collection`**

In [232]:
url = 'https://drive.google.com/uc?id=1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo'

# Read the CSV file
try:
    data = pd.read_csv(url)
except Exception as e:
    print(f"Error reading CSV: {e}")

---
# **Data analysis.**    Аналіз даних.
---

In [233]:
data.sample(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
333,3920000,3000,3,1,2,yes,no,no,no,no,0,no,semi-furnished
167,5285000,4600,2,2,1,yes,no,no,no,yes,2,no,semi-furnished
400,3500000,3512,2,1,1,yes,no,no,no,no,1,yes,unfurnished
487,2870000,5400,4,1,2,yes,no,no,no,no,0,no,unfurnished
385,3570000,3640,2,1,1,yes,no,no,no,no,0,no,unfurnished


In [234]:
data[data.isna()].count()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [235]:
data.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [236]:
x_columns = ["area", "bedrooms","bathrooms"]
y_column = "price"
data_columns = [y_column] + x_columns
data_columns = [y_column, *x_columns]
data_columns

['price', 'area', 'bedrooms', 'bathrooms']

In [237]:
samples = data[data_columns]
samples.sample()

,price,area,bedrooms,bathrooms
150,5600000,5136,3,1


In [238]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


# **Data preparing**

In [239]:
# Визначення колонок
x_columns = ["area", "bedrooms", "bathrooms"]
y_column = "price"

# Формування матриці ознак X і вектору цільових змінних y
X = data[x_columns].values
y = data[y_column].values

# **Data normalizing**

In [240]:
def normalization(data):
  mean = np.mean(data)
  value_range = np.max(data) - np.min(data)
  result = []
  for x in data:
      norm_x = (x-mean)/value_range
      result.append(norm_x)

  return result

In [241]:
norm = pd.DataFrame()

norm['price'] = normalization(data.price)
norm['area'] = normalization(data.area)
norm['bedrooms'] = normalization(data.bedrooms)
norm['bathrooms'] = normalization(data.bathrooms)

norm.head(7)

,price,area,bedrooms,bathrooms
0,0.738811,0.155977,0.206972,0.237920
1,0.647902,0.261818,0.206972,0.904587
2,0.647902,0.330547,0.006972,0.237920
3,0.644872,0.161475,0.206972,0.237920
4,0.575175,0.155977,0.206972,-0.095413
5,0.526690,0.161475,0.006972,0.571254
6,0.466084,0.235702,0.206972,0.571254


In [242]:
X = norm[x_columns].values
y = norm[y_column].values

### Adding an Intercept

To include the intercept \( w_0 \) in our model, we need to add a column of ones to the matrix \( X \). This ensures that the linear regression equation:

$$
y = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_n x_n
$$

is represented as:

$$
y = Xw
$$

where:

- \( X \) is the feature matrix augmented with a column of ones:
  $$
  X = \begin{bmatrix}
  1 & x_{11} & x_{12} & \dots & x_{1n} \\
  1 & x_{21} & x_{22} & \dots & x_{2n} \\
  \vdots & \vdots & \vdots & \ddots & \vdots \\
  1 & x_{m1} & x_{m2} & \dots & x_{mn}
  \end{bmatrix}
  $$
- \( w \) is the coefficient vector, including the intercept \( w_0 \).

In [243]:
X = np.hstack([np.ones((X.shape[0], 1)), X])
X

array([[ 1.        ,  0.15597654,  0.20697248,  0.23792049],
       [ 1.        ,  0.26181847,  0.20697248,  0.90458716],
       [ 1.        ,  0.33054699,  0.00697248,  0.23792049],
       ...,
       [ 1.        , -0.10519184, -0.19302752, -0.09541284],
       [ 1.        , -0.15398909,  0.00697248, -0.09541284],
       [ 1.        , -0.08938428,  0.00697248, -0.09541284]])

---
# **Initializing parameters and starting gradient descent**
---

In [244]:
# Ініціалізація параметрів w
num_features = X.shape[1]
w = np.zeros(num_features)

# Налаштування параметрів алгоритму
learning_rate = 0.01
num_iterations = 1000

# Запуск градієнтного спуску
final_weights, loss_history = gradient_descent(X, y, w, num_iterations, learning_rate=0.001)

print("Оптимальні параметри w:", final_weights)
# print(f"{loss_history=}")

Оптимальні параметри w: [2.04593732e-17 1.27329498e-02 8.56609261e-03 1.37478684e-02]


In [245]:
model = LinearRegression()
model.fit(X, y)
print(f'{model.coef_ = }')
print(f'{model.intercept_ = }')

model.coef_ = array([0.        , 0.47714269, 0.17611257, 0.36001286])
model.intercept_ = 7.014204135081996e-17


---
# **Visualisation**
---